# Import libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
from collections import Counter
import re
import unidecode
import html

# Load the training csv's and combine them into one dataframe

In [2]:
# load all 8 training csv's
df1 = pd.read_csv('reviews/train-1.csv')
df2 = pd.read_csv('reviews/train-2.csv')
df3 = pd.read_csv('reviews/train-3.csv')
df4 = pd.read_csv('reviews/train-4.csv')
df5 = pd.read_csv('reviews/train-5.csv')
df6 = pd.read_csv('reviews/train-6.csv')
df7 = pd.read_csv('reviews/train-7.csv')
df8 = pd.read_csv('reviews/train-8.csv')

# combine the dataframes into one big dataframe
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])

# Show the tail of the data

In [3]:
df.head(10)

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
0,9,B001N2MZT8,903886718,Green Zone [DVD],N,Y,green zone,I found at first it was a little difficult to ...,2010-11-15,1,3,False
1,11,B00GCBVE0Q,282740618,Le secret de Green Knowe,N,Y,NaN,J'ai aimé cette histoire. Les acteurs - et sur...,2014-11-23,2,3,False
2,19,1423165691,883799517,A Disney Sketchbook.,N,N,okay mais...,est-ce une coincidence que la plupart des prin...,2012-12-22,0,0,False
3,33,0061091480,623343977,Your Erroneous Zones,N,N,Arrogant,Wáyné Dyér is á pớpúlár áméricán pérsớnál grớw...,2009-07-21,0,0,True
4,34,B00HZ4CYOY,647510225,König der Mathematik Junior,N,Y,Tớllé Máthé Ápp...,.....unsere Kids mögen diese Art des Lernens. ...,2015-06-01,0,1,False
5,40,B0057YFX9C,240481447,Jason Bourne - Coffret trilogie : La mémoire d...,N,N,Bluray steelbook r&eacute;ussi !,Tr&egrave;s beau coffret a un prix int&eacute;...,2011-10-21,2,3,True
6,46,B007TK77QE,165338194,The Casual Vacancy,N,Y,to long,"To long poor end, is she working up to a follo...",2013-09-28,1,4,True
7,49,B00005JYAX,581643936,Memento,N,N,NaN,"Le film est excellent niveau suspense,... Je l...",2002-09-28,0,3,True
8,55,B004UPIDDC,950170023,Rio,N,N,"Brillant, Majestueux et Fascinant.",Bien sûr que ce n'est pas un film politique ou...,2011-05-07,2,3,True
9,68,B00CSGYLQI,183571812,Disney Princesse - Maison De Poupée - La Reine...,N,Y,Très satisfaite,Château facile à monter et conforme à la descr...,2015-08-31,0,2,False


# Remove row with NaN date value 

In [4]:
df = df.dropna(subset=['review_date'])

# Fix the mixed up product title and review headlines
The fields were switched up sometimes.

- **Note:**  This didn't fix every single row, some products only have one row and we don't know whether those titles and headlines are correct or not
- **Note:**  If one product had 2 reviews and both with the same headline, but one where the headline and title were mixed up, the title and headline of the correct review would be incorrectly switched and be made wrong by this method

In [5]:
def fix_title_headline(df, row):
    
    # get the possible titles of the product id of this row
    rows = df[df['product_id'] == row['product_id']]
    
    # the unique titles of this product
    unique_titles = rows['product_title'].unique()
    
    # get the review headline
    review_headline = row['review_headline']
    
    # check if the review_headline is in the unique titles
    if review_headline in unique_titles:
        
        # replace the review headline of the row with the title
        row['review_headline'] = row['product_title']
        
        # replace the title of the row with the review headline
        row['product_title'] = review_headline
        
    # return the modified row
    return row

In [6]:
# fix the mixed up product titles and headlines
# Note:  This didn't fix every single row,
#        some products only have one row and we don't know whether those titles and headlines are correct or not
# Note:  If one product had 2 reviews and both with the same headline, but one where the headline and title were mixed up, 
#        the title and headline of the correct review would be incorrectly switched and be made wrong by this method
df = df.apply(lambda x: fix_title_headline(df, x), axis=1)

In [7]:
# Unnamed: 0 == 7394 was wrong, but has been fixed now
df.loc[742]

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
742,7701,B00B4VXQEG,819895224,What About Now (Limited Deluxe Edition im Digi...,N,Y,Bớn Jớvi cd,"Véry sátisfiéd with thé púrchásé ớf this cd, i...",2013-04-09,3,6,False
742,7317,B0090CVJZ4,577429472,AmazonBasics Lot de 24 fiches bananes plaquées...,Y,N,Parfaite mais....,Ce lot de fiches bananes est de très bonne fin...,2013-11-04,2,13,True
742,7433,B0001NDG70,796529195,the tarantino connection,N,N,True Romance - Coffret Collector 3 DVD [Éditio...,D'accord Tony scott est un réalisateur ringard...,2004-05-15,2,3,True
742,7394,B00B8SSWJE,491997969,Delta Machine (Deluxe Edition),N,Y,súpér,Áls Fán éin ábsớlútés Múss.<br />Nách dém létz...,2013-04-23,0,6,True
742,7241,2266186671,974337128,L'homme qui voulait être heureux,N,N,Vraiment consternant!!!,Un chapelet de poncifs par chapitre . Un style...,2010-07-14,2,0,True
742,7166,B007ATHNJA,782003376,"Monteverdi, Claudio - L'incoronazione di Poppe...",N,Y,Eine meiner Lieblings - CD´`s,Diese Cd ist ausgesprochener Ohrwurm für Freun...,2013-05-13,0,3,False
742,7554,3125362318,671217915,The Hound of the Baskervilles: Englische Lektü...,N,N,Super spannend,Does the dark night of Dartmoor hide the legen...,2006-03-28,0,24,True
742,7605,B008JGSM6G,345007076,Flow Free,N,Y,addictive,Very addictive game and keeps your mind occupi...,2013-01-19,1,1,False


# Get rid of the html ascii codes
Ascii codes like `&eacute;`

In [8]:
# parse html
df['review_headline'] = df['review_headline'].apply(lambda x: html.unescape(str(x)))
df['review_body'] = df['review_body'].apply(lambda x: html.unescape(str(x)))

# Get rid of the breaklines
Breakline is `<br />`

In [9]:
# remove the breaklines
df['review_headline'] = df['review_headline'].apply(lambda x: str(x).replace('<br />', ' '))
df['review_body'] = df['review_body'].apply(lambda x: str(x).replace('<br />', ' '))

# Get rid of the accent spam
Like: `Ist dié Géilsté Sérié dér Wélt`

In [10]:
def replace_acute_accents(text, accent_map):
    for [accent, char] in accent_map:
        text = re.sub(accent, char, text)
    return text

def remove_accent_spam(text, threshold=0.075):
    
    # the total number of characters
    total = len(text)
    
    # the number of accents
    acutes = 0
    
    # the map of acute char with their non accented variant
    acute_map = np.array([['á', 'a'], ['Á', 'A'], ['é', 'e'], 
                          ['É', 'E'], ['ớ', 'o'], ['ó', 'o'], 
                          ['Ó', 'O'], ['ú', 'u'], ['Ú', 'U']])
    
    # iterate over the characters and check if acute is in their unicode name
    for char in text:
        if char in acute_map[:, 1]:
            return text

    # return the decoded text
    return replace_acute_accents(text, acute_map)

In [11]:
# remove the accent spam
df['review_body'] = df['review_body'].apply(remove_accent_spam)
df['review_headline'] = df['review_headline'].apply(remove_accent_spam)

In [12]:
df.head(10)

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
0,9,B001N2MZT8,903886718,Green Zone [DVD],N,Y,green zone,I found at first it was a little difficult to ...,2010-11-15,1,3,False
1,11,B00GCBVE0Q,282740618,Le secret de Green Knowe,N,Y,nan,J'ai aimé cette histoire. Les acteurs - et sur...,2014-11-23,2,3,False
2,19,1423165691,883799517,A Disney Sketchbook.,N,N,okay mais...,est-ce une coincidence que la plupart des prin...,2012-12-22,0,0,False
3,33,0061091480,623343977,Your Erroneous Zones,N,N,Arrogant,Wayne Dyer is a popular american personal grow...,2009-07-21,0,0,True
4,34,B00HZ4CYOY,647510225,König der Mathematik Junior,N,Y,Tolle Mathe App...,.....unsere Kids mögen diese Art des Lernens. ...,2015-06-01,0,1,False
5,40,B0057YFX9C,240481447,Jason Bourne - Coffret trilogie : La mémoire d...,N,N,Bluray steelbook réussi !,Très beau coffret a un prix intéressant. Boiti...,2011-10-21,2,3,True
6,46,B007TK77QE,165338194,The Casual Vacancy,N,Y,to long,"To long poor end, is she working up to a follo...",2013-09-28,1,4,True
7,49,B00005JYAX,581643936,Memento,N,N,nan,"Le film est excellent niveau suspense,... Je l...",2002-09-28,0,3,True
8,55,B004UPIDDC,950170023,Rio,N,N,"Brillant, Majestueux et Fascinant.",Bien sûr que ce n'est pas un film politique ou...,2011-05-07,2,3,True
9,68,B00CSGYLQI,183571812,Disney Princesse - Maison De Poupée - La Reine...,N,Y,Très satisfaite,Château facile à monter et conforme à la descr...,2015-08-31,0,2,False


In [13]:
# Unnamed: 0 == 7394 was wrong, but has been fixed now
df.loc[742]

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
742,7701,B00B4VXQEG,819895224,What About Now (Limited Deluxe Edition im Digi...,N,Y,Bon Jovi cd,"Very satisfied with the purchase of this cd, i...",2013-04-09,3,6,False
742,7317,B0090CVJZ4,577429472,AmazonBasics Lot de 24 fiches bananes plaquées...,Y,N,Parfaite mais....,Ce lot de fiches bananes est de très bonne fin...,2013-11-04,2,13,True
742,7433,B0001NDG70,796529195,the tarantino connection,N,N,True Romance - Coffret Collector 3 DVD [Éditio...,D'accord Tony scott est un réalisateur ringard...,2004-05-15,2,3,True
742,7394,B00B8SSWJE,491997969,Delta Machine (Deluxe Edition),N,Y,super,Als Fan ein absolutes Muss. Nach dem letzten K...,2013-04-23,0,6,True
742,7241,2266186671,974337128,L'homme qui voulait être heureux,N,N,Vraiment consternant!!!,Un chapelet de poncifs par chapitre . Un style...,2010-07-14,2,0,True
742,7166,B007ATHNJA,782003376,"Monteverdi, Claudio - L'incoronazione di Poppe...",N,Y,Eine meiner Lieblings - CD´`s,Diese Cd ist ausgesprochener Ohrwurm für Freun...,2013-05-13,0,3,False
742,7554,3125362318,671217915,The Hound of the Baskervilles: Englische Lektü...,N,N,Super spannend,Does the dark night of Dartmoor hide the legen...,2006-03-28,0,24,True
742,7605,B008JGSM6G,345007076,Flow Free,N,Y,addictive,Very addictive game and keeps your mind occupi...,2013-01-19,1,1,False


In [14]:
# remove urls

for x in df['review_body'].values:
    try:
        re.sub(r"http\S+", '', x)
    except:
        print(x)

df['review_body'] = df['review_body'].map(lambda x: re.sub(r"http\S+", '', x))

In [15]:
df.head()

,Unnamed: 0,product_id,product_parent,product_title,vine,verified_purchase,review_headline,review_body,review_date,marketplace_id,product_category_id,label
0,9,B001N2MZT8,903886718,Green Zone [DVD],N,Y,green zone,I found at first it was a little difficult to ...,2010-11-15,1,3,False
1,11,B00GCBVE0Q,282740618,Le secret de Green Knowe,N,Y,nan,J'ai aimé cette histoire. Les acteurs - et sur...,2014-11-23,2,3,False
2,19,1423165691,883799517,A Disney Sketchbook.,N,N,okay mais...,est-ce une coincidence que la plupart des prin...,2012-12-22,0,0,False
3,33,0061091480,623343977,Your Erroneous Zones,N,N,Arrogant,Wayne Dyer is a popular american personal grow...,2009-07-21,0,0,True
4,34,B00HZ4CYOY,647510225,König der Mathematik Junior,N,Y,Tolle Mathe App...,.....unsere Kids mögen diese Art des Lernens. ...,2015-06-01,0,1,False


In [16]:
# remove punctuation except for the last one in sentences
df['review_body'] = df['review_body'].map(lambda x: re.sub(r'[\?\.\!\*\/\\\\]+(?=[\?\.\!\*\/\\\\])', '', x))

In [17]:
df['translated_review'] = None

In [19]:
from googletrans import Translator
import textwrap

def translate(a, translator):
    review = str(a['review_body'])
    review_wrapped = textwrap.wrap(review, 5000)

    b = a['translated_review']
    if b is None:
        try:
            translated = ""
            for review_part in review_wrapped:
                translated += translator.translate(review_part, dest='en').text
            return translated
        except:
            return None
    return b

translator = Translator()
df['translated_review'] = df[['review_body', 'translated_review']].apply(lambda a: translate(a, translator), axis=1)

In [21]:
# repeat
df['translated_review'] = df[['review_body', 'translated_review']].apply(lambda a: str(a['review_body']) if translate(a, translator) is None else translate(a, translator), axis=1)

In [22]:
df.to_csv('reviews/train_cleaned.csv')